# Notebook para generar el dataset

Utilizando la biblioteca Spotipy vamos a generar un csv, haciendo una búsqueda general por año para obtener todas las tracks que fueron publicadas. Reduciremos el universo de datos al país Argentina.
Luego, de cada una de las tracks obtenidas se vovlerá a realizar consultas a la Web API de Spotify para obtener información un poco más técnica de cada una de las tracks.
Con todos esos datos, se generará un DataSet y se lo almacenará en un archivo CSV para su uso posterior.

In [1]:
import spotipy #pip install spotipy --upgrade
from spotipy.oauth2 import SpotifyClientCredentials#para gestionar la autenticacion contra la API de Spotify
import pandas as pd
import numpy as np

# Inicializamos Spotipy

##### Credenciales para conectarse a la web api de Spotify

In [2]:
SPOTIPY_CLIENT_ID = 'e8b2379bdd054eb993035179ae40e066'
SPOTIPY_CLIENT_SECRET = 'f191a3a661cc47d68b92a296669dde05'#este valor puede ser cambiado desde la app

##### Incializamos el objeto sp para poder realizar las requests

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)

# Realizamos las consultas por año

##### Seteamos los años en que las tracks fueron publicadas

In [4]:
years = list(range(2000, 2005))#excluye el final
#years = [2000]

##### Seteamos la cantidad de máxima de tracks que queremos tomar como muestra para cada uno de los años

In [5]:
max_cantidad_tracks_anio = 1500

##### Realizamos la consulta

In [6]:
#declaramos un array de diccionarios donde vamos a ir poniendo los resultados, para luego generar el data frame
data = []

#loopeamos por los años seteados
for year in years:
    print('Año: ' + str(year))    
    
    #configuramos la consulta para el año actual
    search_query = 'year:' + str(year)
    
    #incializamos un contador de tracks, para no traer mas de las que configuramos en max_cantidad_tracks_anio
    tracks_obtenidas = 0        
    
    #definimos una cantidad maxima de reintentos ante errores
    max_retry = 3
    
    #la web api solo permite obtener hasta 50 items a la vez, es por eso que para llegar a cantidad max de tracks
    #por año, se deberá realizar más de una consulta para el mismo año
    while tracks_obtenidas < max_cantidad_tracks_anio:                
        
        #si ya no quedan reintentos, se abandona el año
        if(max_retry<=0):
            print('\nSe consumieron todos los reintentos, se abandona el año.')
            break
        
        try:
            #cuando se hace una consulta a la web api, spotify encuentra por ejemplo 120 resultados
            #pero solo permite traer de a 50 (limit) a la vez, para poder traer todos hay que ir moviéndose en ese
            #resultado con el offset. Para traer las primeras 50, el offset = 0, para traer las siguientes 50, el offset = 50
            result = sp.search(search_query, limit=50, offset=tracks_obtenidas, type='track', market='AR')

            if(result is not None and result['tracks'] is not None and len(result['tracks']['items']) > 0):
                #nos quedamos con las tracks
                tracks_results = list(result['tracks']['items'])

                #por cada uno de los tracks obtenidos, consultamos los datos tecnicos
                #y luego generamos una entrada en el array de diccionarios
                for track_result in tracks_results:
                    track_spotify_id = track_result['id']
                    album = track_result['album']
                    artist = track_result['artists'][0]

                    #nos aseguramos de no agregar mas tracks que las que se establecieron en max_cantidad_tracks_anio
                    if(tracks_obtenidas < max_cantidad_tracks_anio):
                        #buscamos en el array de diccionarios 'data' el artista para saber si ya lo tenemos en memoria o
                        #tendremos que hacer una consulta a la web api para obtener su información
                        datos_artista = [x for x in data if x['artista_id']==artist['id']]
                        
                        if(datos_artista is None or datos_artista == []):
                            #si no se tiene aun el artista se lo va a buscar a spotify
                            artista_web_api_result = sp.artist(artist['id'])
                            datos_artista = {'artista_id': artista_web_api_result['id'], 
                                             'artista_generos': ','.join(artista_web_api_result['genres']),
                                             'artista_followers': artista_web_api_result['followers']['total'] if artista_web_api_result['followers'] is not None and artista_web_api_result['followers'] != [] else -1,
                                             'artista_name': artista_web_api_result['name']}
                        else:
                            #hasta este punto 'datos_artista' es un array de diccionarios, tomamos el primer elemento
                            datos_artista = datos_artista[0]


                        #consultamos a la web api de spotify los datos tecnicos de la track actual                    
                        track_features_result = sp.audio_features(track_spotify_id)

                        if(track_features_result is not None and len(track_features_result) > 0):
                            track_features = track_features_result[0]                    

                            #agregamos los datos al array de diccionarios
                            data.append({'popularity': track_result['popularity'], 'id':track_spotify_id, 
                                         'name':track_result['name'], 'album': album['name'], 'duration_ms':track_result['duration_ms'],
                                         'album_release_date': album['release_date'], 'explicit': track_result['explicit'],
                                         'danceability': track_features['danceability'], 'energy': track_features['energy'],
                                         'key': track_features['key'], 'loudness': track_features['loudness'],
                                         'mode': track_features['mode'], 'speechiness': track_features['speechiness'],
                                         'acousticness': track_features['acousticness'], 'instrumentalness': track_features['instrumentalness'],
                                         'liveness': track_features['liveness'], 'valence': track_features['valence'],
                                         'tempo': track_features['tempo'], 'artista_id': datos_artista['artista_id'],
                                         'artista_generos': datos_artista['artista_generos'], 'artista_followers': datos_artista['artista_followers'],
                                         'artista_name': datos_artista['artista_name']
                                        })

                            tracks_obtenidas += 1
                            print(str(tracks_obtenidas) + ' tracks obtenidas!', end='\r')

            else:
                print('Datos vacíos, reintentando')
                #ante cualquier error, bajamos la cantidad de reintentos y seguimos
                max_retry -= 1                    
        
        except Exception as e: 
            print('Error: '+ str(e))
            print('Reintentando')
            #ante cualquier error, bajamos la cantidad de reintentos y seguimos
            max_retry -=1
    print('\nAño completado')
            

Año: 2000
1500 tracks obtenidas!
Año completado
Año: 2001
1500 tracks obtenidas!
Año completado
Año: 2002
1500 tracks obtenidas!
Año completado
Año: 2003
1500 tracks obtenidas!
Año completado
Año: 2004
Error: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)
Reintentando
1500 tracks obtenidas!
Año completado


# Generamos el DataFrame

In [7]:
df_tracks = pd.DataFrame(data)

In [8]:
df_tracks.shape

(7500, 21)

In [9]:
df_tracks.head()

,popularity,id,name,album,album_release_date,explicit,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name
0,64,6rg1MBZqggsQ5olFGTw0rr,Spaghetti del Rock,"Narigón del Siglo, Yo Te Dejo Perfumado en la ...",2000-02-14,False,0.626,0.389,4,-11.609,...,0.0254,0.67700,0.000011,0.117,0.656,97.708,6ZIgPKHzpcswB8zh7sRIhx,"argentine rock,latin alternative,latin rock,ro...",567708,Divididos
1,58,6KzeUBu1BE8q24CblG6oku,Tramposa Y Mentirosa,Un Homenaje Al Cielo,2000-05-08,False,0.562,0.756,11,-4.541,...,0.0487,0.21700,0.000000,0.150,0.841,87.771,2Mu8h5sFkOziL0Rfn7FXIA,"argentine rock,cumbia santafesina,cumbia villera",514120,Leo Mattioli
2,59,0NELwHhQu4EVcmGmu8D7vV,Toco Y Me Voy,Hijos Del Culo,2000-01-01,False,0.641,0.676,5,-5.627,...,0.0351,0.32600,0.000000,0.101,0.461,89.991,6MxyNXnnmwQwdW2PD0gXYO,"argentine rock,latin alternative,latin rock,ro...",483153,Bersuit Vergarabat
3,58,1g0mhq4ZDQE3cTd1XLcYVf,Sapo de Otro Pozo,Fulanos de Nadie,2000-06-27,False,0.632,0.771,7,-5.063,...,0.0353,0.50300,0.000000,0.141,0.723,113.230,6Iv9dXeKX45ff7qe0LDuFW,"argentine rock,latin alternative,latin rock,ro...",194389,Los Caballeros De La Quema
4,81,3AJwUDP919kvQ9QcozQPxg,Yellow,Parachutes,2000-07-10,False,0.429,0.661,11,-7.227,...,0.0281,0.00239,0.000130,0.234,0.285,173.365,4gzpq5DPGxSnKTe4SA8HAU,"permanent wave,pop",25933581,Coldplay


# Generamos el CSV

In [10]:
tracks_csv_name = 'tracks.csv'

In [11]:
df_tracks.to_csv(tracks_csv_name,index=False,encoding='utf-8')